In [0]:
#Relevant Links
#1. https://towardsdatascience.com/bert-classifier-just-another-pytorch-model-881b3cf05784


In [0]:
! pip install constant
!pip install bert-pytorch
!pip install pytorch-pretrained-bert pytorch-nlp
!pip install -U -q PyDrive

In [0]:
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, recall_score, precision_score
from sklearn.utils.class_weight import compute_class_weight
from keras.layers import Input, Dense, Dropout
from keras.models import Model
import pandas as pd
import numpy as np
import os, sys
from google.colab import drive

sys.path.append(os.path.join(os.path.dirname(sys.path[0]), 'analysis'))
sys.path.append(os.path.join(os.path.dirname(os.path.dirname(sys.path[0])), 'configs' ))

import constant

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import warnings
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification, BertAdam, BertModel
from pytorch_pretrained_bert import BertConfig
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


'Tesla P100-PCIE-16GB'

In [0]:
#Data Reading
# Mount drive for data reading
# This will prompt for authorization.
drive.mount('/content/drive')
path = "/content/drive/Shared drives/CIS 519 Project/Code/Dataset/Resampled4/"
torch.cuda.empty_cache()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!nvidia-smi

Sun Dec  8 17:32:43 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    25W / 250W |     10MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
#Read full data path = "/content/drive/Shared drives/CIS 519 Project/Code/Dataset/"
df_train_total = pd.read_csv(path + 'train.csv')
df_val_total = pd.read_csv(path + 'validation.csv')

In [0]:
df_train_total.head(10)

,id,target,comment_text,female,male,homosexual_gay_or_lesbian,christian,jewish,muslim,black,white,psychiatric_or_mental_illness,identity_annotator_count,toxicity_annotator_count,created_date
0,5367475,0.500000,This is an insult to all immigrants who took t...,0.800000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.1,10,10,2017-06-06 17:53:11.976493+00
1,389581,0.166667,"Eric, I was unaware that Obama had a line of p...",0.000000,0.600000,0.000000,0.0,0.0,0.0,0.000000,0.9,0.0,10,6,2016-07-15 23:22:57.603863+00
2,259203,0.600000,"""malign her character because of alleged illeg...",1.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,4,10,2016-03-26 19:14:41.102508+00
3,6274158,0.000000,"Couldn't agree more with your last sentence, E...",0.000000,0.700000,0.900000,0.7,0.0,0.0,0.000000,0.0,0.0,10,4,2017-11-02 15:49:53.222384+00
4,5435081,0.500000,"The liberal, married, gay Black woman who took...",0.833333,0.166667,0.833333,0.0,0.0,0.0,0.833333,0.0,0.0,6,10,2017-06-18 13:26:29.297687+00
5,5693877,0.000000,"Get over it, two men took to the ring and one ...",0.000000,1.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,4,4,2017-08-01 02:59:30.400500+00
6,5801144,0.600000,"Yes, yes, it's ""The Media"" who forced Trump to...",1.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,1.0,0.0,4,10,2017-08-19 16:02:37.201001+00
7,5078252,0.500000,"""cannon fodder"" - they are ""acceptable losses""...",1.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,4,10,2017-03-31 18:03:36.139268+00
8,5277082,0.000000,"No, not who cares, as in who cares, rather htt...",0.000000,1.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,4,4,2017-05-19 14:33:37.520110+00
9,489731,0.828571,The hate of white males from the progressive p...,0.000000,1.000000,0.000000,0.0,0.0,0.0,0.000000,1.0,0.0,4,70,2016-09-24 04:44:47.907794+00


In [0]:
df_val_total.head(10)

,id,target,comment_text,female,male,homosexual_gay_or_lesbian,christian,jewish,muslim,black,white,psychiatric_or_mental_illness,identity_annotator_count,toxicity_annotator_count,created_date
0,400154,0.000000,Donald always has been a lefty wing Manhattani...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,4,2016-07-24 22:43:28.495610+00
1,5092755,0.000000,"Again, to be fair, it was just the one guy who...",0.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,4,2017-04-04 20:08:40.823984+00
2,797853,0.600000,I am going to start an organization so ALL ord...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4,10,2017-01-06 04:54:33.684047+00
3,5046660,1.000000,LMAO...what on god's green earth makes you thi...,0.1,0.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,4,2017-03-23 23:54:30.927369+00
4,882416,0.000000,"Master Yoda told me; Hmmmm...Hmmm.. ""Much whin...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,4,2017-01-23 04:17:54.377161+00
5,984833,0.166667,I am SO happy and grateful to see people stand...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,6,2017-02-12 07:08:51.852690+00
6,5499900,0.166667,Trump has been objectifying women for decades....,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,6,2017-06-29 19:30:21.743759+00
7,728632,0.500000,Ms. Kurdi claims that Canada is partly respons...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,10,2016-12-19 15:24:50.573840+00
8,5408070,0.800000,The Catholic church confuses forgiveness with ...,0.0,0.6,0.0,1.0,0.0,0.0,0.0,0.0,0.0,10,10,2017-06-13 18:26:08.994210+00
9,5253827,0.000000,"""Cultural Appropriation"" as an accusation is a...",0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4,4,2017-05-14 02:35:25.722340+00


In [0]:
# comments = pd.DataFrame(df.comment_text)
# #comments.to_csv(path + "only_comments.csv")
# comments_text = df.comment_text
# df_store = df.copy()
# comments_store = comments.copy()

In [0]:
df_train = df_train_total#.sample(5000)
df_val = df_val_total#.sample(5000)

In [0]:
comments_train = df_train.comment_text
comments_val = df_val.comment_text

In [0]:
#Generate protected attribute - gender
def extract_female_gender(x):
  if np.isnan(x.female) or x.female < 0.5:
      return 0
  else:
    return 1

In [0]:
#Generate unprotected attribute labels
def get_unprotected_class(list_of_protected):
  new = [1 if i == 0 else 0 for i in list_of_protected]
  return new

In [0]:
#Calculate metrics
def get_metrics(labels, preds):
  pred_flat = preds.flatten()
  labels_flat = labels.flatten()

  acc = accuracy_score(labels_flat, pred_flat)
  pre = precision_score(labels_flat, pred_flat)
  rec = recall_score(labels_flat, pred_flat)
  f1 = f1_score(labels_flat, pred_flat, average="weighted")

  return acc, pre, rec, f1

In [0]:
#Generate labels
toxicity_labels_train = list(df_train.target.apply(lambda x: 1 if x >= 0.5 else 0))
identity_labels_train = list(df_train.apply(extract_female_gender, axis = 1))
toxicity_labels_val = list(df_val.target.apply(lambda x: 1 if x >= 0.5 else 0))
identity_labels_val = list(df_val.apply(extract_female_gender, axis = 1))
unprotected_labels_train = get_unprotected_class(identity_labels_train)
unprotected_labels_val = get_unprotected_class(identity_labels_val)

In [0]:
print(len(comments_train), len(toxicity_labels_train))
print(comments_train[:10])
print(toxicity_labels_train[:10])
print(identity_labels_train[:10])

10974 10974
0    This is an insult to all immigrants who took t...
1    Eric, I was unaware that Obama had a line of p...
2    "malign her character because of alleged illeg...
3    Couldn't agree more with your last sentence, E...
4    The liberal, married, gay Black woman who took...
5    Get over it, two men took to the ring and one ...
6    Yes, yes, it's "The Media" who forced Trump to...
7    "cannon fodder" - they are "acceptable losses"...
8    No, not who cares, as in who cares, rather htt...
9    The hate of white males from the progressive p...
Name: comment_text, dtype: object
[1, 0, 1, 0, 1, 0, 1, 1, 0, 1]
[1, 0, 1, 0, 1, 0, 1, 1, 0, 0]


In [0]:
MAX_SEQUENCE_LENGTH = 128
SEED = 519
BATCH_SIZE = 32
BERT_MODEL_PATH = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case=True)

In [0]:
def convert_lines(example, max_seq_length,tokenizer):
    #max_seq_length -=2
    all_tokens = []
    longer = 0
    for text in tqdm(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[:max_seq_length]
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
        all_tokens.append(one_token)
    print("Tokens longer than max_length: ", longer)
    return np.array(all_tokens)

In [0]:
#Prepare data
input_train = convert_lines(comments_train.fillna("DUMMY_VALUE"), MAX_SEQUENCE_LENGTH, tokenizer)
toxicity_labels_train = torch.tensor(toxicity_labels_train)
female_labels_train = torch.tensor(identity_labels_train)

input_val = convert_lines(comments_val.fillna("DUMMY_VALUE"), MAX_SEQUENCE_LENGTH, tokenizer)
toxicity_labels_val = torch.tensor(toxicity_labels_val)
female_labels_val = torch.tensor(identity_labels_val)


  0%|          | 0/2000 [00:00<?, ?it/s]

Tokens longer than max_length:  2690


100%|██████████| 2000/2000 [00:02<00:00, 930.35it/s]


Tokens longer than max_length:  544


In [0]:
print(torch.sum(toxicity_labels_train).data)
print(torch.sum(female_labels_train).data)

print(torch.sum(toxicity_labels_val).data)
print(torch.sum(female_labels_val).data)


tensor(5424)
tensor(5414)
tensor(338)
tensor(1144)


In [0]:
#Data Loader
X_train = torch.utils.data.TensorDataset(torch.tensor(input_train, dtype=torch.long), toxicity_labels_train, female_labels_train)
train_loader = torch.utils.data.DataLoader(X_train, batch_size=32, shuffle=True)
#tk0 = tqdm(train_loader)

X_val = torch.utils.data.TensorDataset(torch.tensor(input_val, dtype=torch.long), toxicity_labels_val, female_labels_val)
val_loader = torch.utils.data.DataLoader(X_val, batch_size=32, shuffle=True)
#vk0 = tqdm(val_loader)


In [0]:
#FAIRNESS METRICS FUNCTION

def get_fairness_metrics(actual_labels, y_pred, protected_labels, non_protected_labels, thres):

  def get_toxicity_rates(y_pred, protected_labels, non_protected_labels, thres):
    protected_ops = y_pred[protected_labels == 1]
    protected_prob = sum(protected_ops)/len(protected_ops)

    non_protected_ops = y_pred[non_protected_labels == 1]
    non_protected_prob = sum(non_protected_ops)/len(non_protected_ops)

    return np.round(protected_prob, 2), np.round(non_protected_prob, 2)

  def get_true_positive_rates(actual_labels, y_pred, protected_labels, non_protected_labels, thres):

    protected_ops = y_pred[np.bitwise_and(protected_labels == 1, actual_labels == 1)]
    protected_prob = sum(protected_ops)/len(protected_ops)

    non_protected_ops = y_pred[np.bitwise_and(non_protected_labels == 1, actual_labels == 1)]
    non_protected_prob = sum(non_protected_ops)/len(non_protected_ops)

    return np.round(protected_prob, 2), np.round(non_protected_prob, 2)


  def get_false_positive_rates(actual_labels, y_pred, protected_labels, non_protected_labels, thres):

    protected_ops = y_pred[np.bitwise_and(protected_labels == 1, actual_labels ==0)]
    protected_prob = sum(protected_ops)/len(protected_ops)

    non_protected_ops = y_pred[np.bitwise_and(non_protected_labels == 1, actual_labels == 0)]
    non_protected_prob = sum(non_protected_ops)/len(non_protected_ops)

    return np.round(protected_prob, 2), np.round(non_protected_prob, 2)

  def demographic_parity(y_pred, protected_labels, non_protected_labels, thres):

    protected_ops = y_pred[protected_labels == 1]
    protected_prob = sum(protected_ops)/len(protected_ops)

    non_protected_ops = y_pred[non_protected_labels == 1]
    non_protected_prob = sum(non_protected_ops)/len(non_protected_ops)

    return abs(protected_prob - non_protected_prob) #later take absolute diff - but we want to show females predicted more toxic than male

  # | P_female(C = 1| Y = 1) - P_male(C = 1 | Y = 1) | < thres
  def true_positive_parity(actual_labels, y_pred, protected_labels, non_protected_labels, thres):

    protected_ops = y_pred[np.bitwise_and(protected_labels == 1, actual_labels == 1)]
    protected_prob = sum(protected_ops)/len(protected_ops)

    non_protected_ops = y_pred[np.bitwise_and(non_protected_labels == 1, actual_labels == 1)]
    non_protected_prob = sum(non_protected_ops)/len(non_protected_ops)

    return abs(protected_prob - non_protected_prob) #later take absolute diff - but we want to show females predicted more toxic than male

  # | P_female(C = 1| Y = 0) - P_male(C = 1 | Y = 0) | < thres
  def false_positive_parity(actual_labels, y_pred, protected_labels, non_protected_labels, thres):

    protected_ops = y_pred[np.bitwise_and(protected_labels == 1, actual_labels ==0)]
    protected_prob = sum(protected_ops)/len(protected_ops)

    non_protected_ops = y_pred[np.bitwise_and(non_protected_labels == 1, actual_labels == 0)]
    non_protected_prob = sum(non_protected_ops)/len(non_protected_ops)

    return abs(protected_prob - non_protected_prob) #later take absolute diff - but we want to show females predicted more toxic than male


  # Satisfy both true positive parity and false positive parity
  def equalized_odds(actual_labels, y_pred, protected_labels, non_protected_labels, thres):
    return true_positive_parity(actual_labels, y_pred, protected_labels, non_protected_labels, thres) + false_positive_parity(actual_labels, y_pred, protected_labels, non_protected_labels, thres)

  female_tox_rate, nf_tox_rate = get_toxicity_rates(y_pred, protected_labels, non_protected_labels, thres)
  female_tp_rate, nf_tp_rate = get_true_positive_rates(actual_labels, y_pred, protected_labels, non_protected_labels, thres)
  female_fp_rate, nf_fp_rate = get_false_positive_rates(actual_labels, y_pred, protected_labels, non_protected_labels, thres)
  demo_parity = demographic_parity(y_pred, protected_labels, non_protected_labels, thres)
  tp_parity = true_positive_parity(actual_labels, y_pred, protected_labels, non_protected_labels, thres)
  fp_parity = false_positive_parity(actual_labels, y_pred, protected_labels, non_protected_labels, thres)
  equ_odds = equalized_odds(actual_labels, y_pred, protected_labels, non_protected_labels, thres)

  return female_tox_rate, nf_tox_rate, female_tp_rate, nf_tp_rate, female_fp_rate, nf_fp_rate, demo_parity, tp_parity, fp_parity, equ_odds



In [0]:
"""
FairClassifier - BERT + equipped with adversarial network - PyTorch Implementation
"""
config = BertConfig(vocab_size_or_config_json_file=32000, hidden_size=768,
        num_hidden_layers=12, num_attention_heads=12, intermediate_size=3072, 
        hidden_dropout_prob=0.1)

class Classifier(nn.Module):
    def __init__(self, toxicity_labels = 2):
        super(Classifier, self).__init__()
        
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.c1 = nn.Linear(config.hidden_size, 324)
        #self.c2 = nn.Linear(config.intermediate_size, 324)
        self.c3 = nn.Linear(324, toxicity_labels)

        nn.init.xavier_normal_(self.c1.weight)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):

        
        #BERT
        _, pooled_output = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
        pooled_output = self.dropout(pooled_output)

        # Classifier
        classifier_prev_output = F.relu(self.c1(pooled_output))
        #classifier_prev_output = F.relu(self.c2(classifier))
        classifier_output = self.c3(classifier_prev_output)

        return classifier_output, classifier_prev_output

class Adversary(nn.Module):
    def __init__(self, identity_labels = 2):
        super(Adversary, self).__init__()

        self.a1 = nn.Linear(324,120)
        self.a2 = nn.Linear(120, identity_labels)

        nn.init.xavier_normal_(self.a1.weight)

    def forward(self, input_ids):


        #Adversary
        adversary = F.relu(self.a1(input_ids))
        adversary_output = self.a2(adversary)

        return adversary_output




In [0]:
def conduct_validation(net, data_loader, adv = False):

    eval_loss, eval_accuracy, eval_precision, eval_recall, eval_f1 = 0, 0, 0, 0, 0
    nb_eval_steps = 0
    
    predictions_net = np.empty((0,))
    truths = np.empty((0,))
    identities = np.empty((0,))
    correct_net = 0
    total = 0

    net.eval()
    with torch.no_grad(): # IMPORTANT: we don't want to do back prop during validation/testing!
      for index, data in enumerate(data_loader):

        text, toxic_truth, female_truth = data

        text = text.to(device)
        toxic_truth = toxic_truth.to(device)
        female_truth = female_truth.to(device)

        if adv:
          net_outputs, net_prev_outputs = net(text)
        else:
          net_outputs = net(text)
        _, net_predicted = torch.max(net_outputs.data, 1)

        batch_size = toxic_truth.size(0)
        total += batch_size
        correct_net_batch = (net_predicted == toxic_truth).sum().item()
        correct_net += correct_net_batch

        
        predictions_net = np.concatenate((predictions_net, net_predicted.cpu().numpy()))
        truths = np.concatenate((truths, toxic_truth.cpu().numpy()))
        identities = np.concatenate((identities, female_truth.cpu().numpy()))

        pred = net_predicted.detach().cpu().numpy()
        label_ids = toxic_truth.to('cpu').numpy()

        tmp_eval_accuracy, tmp_eval_precision, temp_eval_recall, tmp_eval_f1 = get_metrics(label_ids, pred)

        eval_accuracy += tmp_eval_accuracy
        eval_precision += tmp_eval_precision
        eval_recall += temp_eval_recall
        eval_f1 += tmp_eval_f1
        nb_eval_steps += 1

    f1_score = eval_f1/nb_eval_steps
    prec_score = eval_precision/nb_eval_steps
    recall_score = eval_recall/nb_eval_steps
    acc_score = eval_accuracy/nb_eval_steps

    print("F1 Score: ", f1_score)
    print("Precision Score: ", prec_score)
    print("Recall Score: ", recall_score)
    print("Acc Score: ", acc_score, "\n\n")

    net.train()
    
    return (predictions_net, truths, identities, acc_score)

In [0]:
def pretrain_classifier(clf, optimizer_clf, train_loader, loss_criterion, epochs):

  pretrain_classifier_loss = 0
  steps = 0

  for epoch in range(epochs):

    print("Epoch: ", epoch + 1)
    epoch_loss = 0
    epoch_batches = 0

    for i, data in enumerate(train_loader): # starting from the 0th batch
        # get the inputs and labels
        inputs, toxicity_true, female_true = data
        inputs = inputs.to(device)
        # toxicity_true = torch.sparse.torch.eye(2).index_select(dim=0, index=toxicity_true) 
        # female_true = torch.sparse.torch.eye(2).index_select(dim=0, index=female_true) 
        toxicity_true = toxicity_true.to(device)
        female_true = female_true.to(device)

        optimizer_clf.zero_grad()

        classifier_output, _ = clf(inputs)
        classifier_loss = loss_criterion(classifier_output, toxicity_true) # compute loss
        classifier_loss.backward() # back prop
        optimizer_clf.step()
        pretrain_classifier_loss += classifier_loss.item()
        epoch_loss += classifier_loss.item()
        epoch_batches += 1
        steps += 1

    print("Average Pretrain Classifier epoch loss: ", epoch_loss/epoch_batches)
  print("Average Pretrain Classifier batch loss: ", pretrain_classifier_loss/steps)

  return clf


def pretrain_adversary(adv, clf, optimizer_adv, train_loader, loss_criterion, epochs):
  
  pretrain_adversary_loss = 0
  steps = 0

  for epoch in range(epochs):

    print("Epoch: ", epoch + 1)
    epoch_loss = 0
    epoch_batches = 0
    for i, data in enumerate(train_loader): # starting from the 0th batch
        # get the inputs and labels
        inputs, toxicity_true, female_true = data
        inputs = inputs.to(device)
        # toxicity_true = torch.sparse.torch.eye(2).index_select(dim=0, index=toxicity_true) 
        # female_true = torch.sparse.torch.eye(2).index_select(dim=0, index=female_true) 
        toxicity_true = toxicity_true.to(device)
        female_true = female_true.to(device)

        optimizer_adv.zero_grad()

        _, classifier_prev_output = clf(inputs)
        adversary_output = adv(classifier_prev_output)
        adversary_loss = loss_criterion(adversary_output, female_true) # compute loss
        adversary_loss.backward() # back prop
        optimizer_adv.step()
        pretrain_adversary_loss += adversary_loss.item()
        epoch_loss += adversary_loss.item()
        epoch_batches += 1
        steps += 1

    print("Average Pretrain Adversary epoch loss: ", epoch_loss/epoch_batches)
  print("Average Pretrain Adversary batch loss: ", pretrain_adversary_loss/steps)

  return adv


def train_adversary(adv, clf, optimizer_adv, train_loader, loss_criterion, epochs=1):
  
  adv_loss = 0
  steps = 0

  for epoch in range(epochs):
    for i, data in enumerate(train_loader): # starting from the 0th batch
        # get the inputs and labels
        inputs, toxicity_true, female_true = data
        inputs = inputs.to(device)
        # toxicity_true = torch.sparse.torch.eye(2).index_select(dim=0, index=toxicity_true) 
        # female_true = torch.sparse.torch.eye(2).index_select(dim=0, index=female_true) 
        toxicity_true = toxicity_true.to(device)
        female_true = female_true.to(device)

        optimizer_adv.zero_grad()

        classifier_output, classifier_prev_output = clf(inputs)
        adversary_output = adv(classifier_prev_output)
        adversary_loss = loss_criterion(adversary_output, female_true) # compute loss
        adversary_loss.backward() # back prop
        optimizer_adv.step()
        adv_loss += adversary_loss.item()
        steps += 1
  
  print("Average Adversary batch loss: ", adv_loss/steps)

  return adv

def train_classifier(clf, optimizer_clf, adv, train_loader, loss_criterion, lbda):

  for i, data in enumerate(train_loader): # starting from the 0th batch
      # get the inputs and labels
      inputs, toxicity_true, female_true = data
      inputs = inputs.to(device)
      # toxicity_true = torch.sparse.torch.eye(2).index_select(dim=0, index=toxicity_true) 
      # female_true = torch.sparse.torch.eye(2).index_select(dim=0, index=female_true) 
      toxicity_true = toxicity_true.to(device)
      female_true = female_true.to(device)

      # Toxic classifier part

      optimizer_clf.zero_grad()

      classifier_output, classifier_prev_output = clf(inputs)
      adversary_output = adv(classifier_prev_output)
      adversary_loss = loss_criterion(adversary_output, female_true)
      classifier_loss = loss_criterion(classifier_output, toxicity_true) # compute loss
      total_classifier_loss = classifier_loss - lbda * adversary_loss
      total_classifier_loss.backward() # back prop
      
      optimizer_clf.step()

      print("Adversary Mini-Batch loss: ", adversary_loss.item())
      print("Classifier Mini-Batch loss: ", classifier_loss.item())
      print("Total Mini-Batch loss: ", total_classifier_loss.item())

      break

  return clf
      


In [0]:
#NEW IMPLEMENTATION WITH PRETRAINING

# Training Process

lambda_params = [3]

lbda_train_accs = []
lbda_valid_accs = []
protected_toxicity_rates = []
unprotected_toxicity_rates = []
protected_tp_rates = []
unprotected_tp_rates = []
protected_fp_rates = []
unprotected_fp_rates = []
demo_parity_scores = []
tp_parity_scores = []
fp_parity_scores = []
equ_odds_scores = []

for lbda in lambda_params:

  #DEFINING MODELS

  clf = Classifier(toxicity_labels = 2) # instantiate the nn
  adv = Adversary(identity_labels = 2)

  loss_criterion = torch.nn.CrossEntropyLoss()

  # Defining optimizers
  optimizer_adv = optim.Adam(adv.parameters(), lr=0.001)

  lrlast = .001
  lrmain = .00001
  optimizer_clf = optim.Adam(
      [
          {"params":clf.bert.parameters(),"lr": lrmain},
          {"params":clf.c1.parameters(), "lr": lrlast},
          #{"params":clf.c2.parameters(), "lr": lrlast},
      {"params":clf.c3.parameters(), "lr": lrlast}    
    ])

  clf.to(device)
  adv.to(device)

  #PRETRAIN CLASSIFIER

  for param in adv.parameters():
    param.requires_grad = False

  clf = pretrain_classifier(clf, optimizer_clf, train_loader, loss_criterion, 3)

  for param in adv.parameters():
    param.requires_grad = True

  #PRETRAIN ADVERSARY

  for param in clf.parameters():
    param.requires_grad = False

  adv = pretrain_adversary(adv, clf, optimizer_adv, train_loader, loss_criterion, 3)

  for param in clf.parameters():
    param.requires_grad = True

  print('Lambda: ' + str(lbda))

  train_accs = []
  valid_accs = []
  iterations = 20

  for iteration in range(iterations):  # loop over the dataset multiple times
      print("Iteration: ", iteration)

      #TRAIN ADVERSARY FOR 1 EPOCH

      for param in clf.parameters():
        param.requires_grad = False

      adv = train_adversary(adv, clf, optimizer_adv, train_loader, loss_criterion, epochs=1)

      for param in clf.parameters():
        param.requires_grad = True

      #TRAIN CLASSIFIER FOR 1 SAMPLE MINI BATCH

      for param in adv.parameters():
        param.requires_grad = False

      clf = train_classifier(clf, optimizer_clf, adv, train_loader, loss_criterion, lbda)

      for param in adv.parameters():
        param.requires_grad = True

      if (iteration + 1) % 2 == 0:
        
        print('Training metrics:')
        y_pred, actual_labels, protected_labels, acc_score = conduct_validation(clf, train_loader, adv = True)
        train_accs.append(acc_score)

        print("\n")
        print("Fairness Metrics on Train:")
        non_protected_labels = np.asarray(get_unprotected_class(protected_labels))
        thres = 0.5
        female_tox_rate, nf_tox_rate, female_tp_rate, nf_tp_rate, female_fp_rate, nf_fp_rate, demo_parity, tp_parity, fp_parity, equ_odds =\
        get_fairness_metrics(actual_labels, y_pred, protected_labels, non_protected_labels, thres)

        print("Toxicity Prediction Rates: ", "Female -", female_tox_rate, "Non-Female - ", nf_tox_rate)
        print("True Positive Prediction Rates: ", "Female -", female_tp_rate, "Non-Female - ", nf_tp_rate)
        print("False Positive Prediction Rates: ", "Female -", female_fp_rate, "Non-Female - ", nf_fp_rate)
        print("Demographic Parity: ", demo_parity)
        print("True Positive Parity: ", tp_parity)
        print("False Positive Parity: ", fp_parity)
        print("Equalized Odds: ", equ_odds)
        print("\n")
        print('Validation metrics:')
        y_pred, actual_labels, protected_labels, acc_score = conduct_validation(clf, val_loader, adv = True)
        valid_accs.append(acc_score)
        
        print("\n")
        print("Fairness Metrics on Validation:")
        non_protected_labels = np.asarray(get_unprotected_class(protected_labels))
        thres = 0.5
        female_tox_rate, nf_tox_rate, female_tp_rate, nf_tp_rate, female_fp_rate, nf_fp_rate, demo_parity, tp_parity, fp_parity, equ_odds =\
        get_fairness_metrics(actual_labels, y_pred, protected_labels, non_protected_labels, thres)

        print("Toxicity Prediction Rates: ", "Female -", female_tox_rate, "Non-Female - ", nf_tox_rate)
        print("True Positive Prediction Rates: ", "Female -", female_tp_rate, "Non-Female - ", nf_tp_rate)
        print("False Positive Prediction Rates: ", "Female -", female_fp_rate, "Non-Female - ", nf_fp_rate)
        print("Demographic Parity: ", demo_parity)
        print("True Positive Parity: ", tp_parity)
        print("False Positive Parity: ", fp_parity)
        print("Equalized Odds: ", equ_odds)
        print("\n\n\n__________________")

        if iteration == iterations -1:
          protected_toxicity_rates.append(female_tox_rate)
          unprotected_toxicity_rates.append(nf_tox_rate)
          protected_tp_rates.append(female_tp_rate)
          unprotected_tp_rates.append(nf_tp_rate)
          protected_fp_rates.append(female_fp_rate)
          unprotected_fp_rates.append(nf_fp_rate)
          demo_parity_scores.append(demo_parity)
          tp_parity_scores.append(tp_parity)
          fp_parity_scores.append(fp_parity)
          equ_odds_scores.append(equ_odds)

  lbda_train_accs.append(train_accs)  
  lbda_valid_accs.append(valid_accs)

  # del clf
  # del adv

  # torch.cuda.empty_cache()

print('Finished Training')



100%|██████████| 407873900/407873900 [00:33<00:00, 12348685.71B/s]

Epoch:  1
Average Pretrain Classifier epoch loss:  0.5526828840592165
Epoch:  2
Average Pretrain Classifier epoch loss:  0.34685237462423285
Epoch:  3
Average Pretrain Classifier epoch loss:  0.26637104820492663
Average Pretrain Classifier batch loss:  0.3886354356294587
Epoch:  1
Average Pretrain Adversary epoch loss:  0.35933143004731605
Epoch:  2
Average Pretrain Adversary epoch loss:  0.23223073587212548
Epoch:  3
Average Pretrain Adversary epoch loss:  0.21500630925461084
Average Pretrain Adversary batch loss:  0.2688561583913508
Lambda: 3
Iteration:  0
Average Adversary batch loss:  0.19585806231283237
Adversary Mini-Batch loss:  0.3093579411506653
Classifier Mini-Batch loss:  0.28543177247047424
Total Mini-Batch loss:  -0.6426420211791992
Iteration:  1
Average Adversary batch loss:  0.29629818367454125
Adversary Mini-Batch loss:  0.3134836256504059
Classifier Mini-Batch loss:  0.1830923855304718
Total Mini-Batch loss:  -0.7573585510253906
Training metrics:
F1 Score:  0.930783780

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


F1 Score:  0.8101646904543559
Precision Score:  0.4256701463284097
Recall Score:  0.8389014865205342
Acc Score:  0.7827380952380952 




Fairness Metrics on Validation:
Toxicity Prediction Rates:  Female - 0.31 Non-Female -  0.37
True Positive Prediction Rates:  Female - 0.82 Non-Female -  0.88
False Positive Prediction Rates:  Female - 0.21 Non-Female -  0.26
Demographic Parity:  0.06467714528462193
True Positive Parity:  0.060731319554848895
False Positive Parity:  0.05280512387696404
Equalized Odds:  0.11353644343181293



__________________
Finished Training


In [0]:
torch.save(clf.state_dict(), path + "SC_Classifier_Final_AllData")
torch.save(adv.state_dict(), path + "SC_Adversary_Final_AllData")

In [0]:
del adv
del clf
torch.cuda.empty_cache()